<a href="https://colab.research.google.com/github/AhnJG/AI-Project/blob/master/Colab/0705__EfficientNetB7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Lib

In [1]:
!pip install tf-nightly

     |████████████████████████████████| 322.9MB 45kB/s 
     |████████████████████████████████| 3.0MB 42.7MB/s 
     |████████████████████████████████| 460kB 54.2MB/s 


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import pandas as pd
from sklearn.model_selection import train_test_split
import os
print(tf.__version__)

2.4.0-dev20200706


# Data Load

In [3]:
# Data Load
# x_data = np.load('/content/drive/My Drive/dacon_voice/data/np_save/preprocessed_melspec_train.npy')
x_data = np.load('/content/drive/My Drive/dacon_voice/data/np_save/stacked_melspec_train_96.npy')
y_source = pd.read_csv('/content/drive/My Drive/dacon_voice/data/train_answer.csv', index_col=0)
y_data = y_source.values

# input_shape = (128, 128, 3)
input_shape = x_data.shape[1:]

print("x_train.shape : ", x_data.shape)
print("input_shape : ", input_shape)
print("y_data.shape : ", y_data.shape)

x_train.shape :  (100000, 96, 96, 3)
input_shape :  (96, 96, 3)
y_data.shape :  (100000, 30)


# Model

In [4]:
model_path = '/content/drive/My Drive/dacon_voice/model/20200706_EfficientNetB7/'
model = Sequential()
model.add(tf.keras.applications.EfficientNetB7(include_top=True, weights=None, input_shape=input_shape, pooling=None, classes=30))

In [5]:
model.compile(loss=tf.keras.losses.KLDivergence(), optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb7 (Functional)  (None, 30)                64174517  
Total params: 64,174,517
Trainable params: 63,863,790
Non-trainable params: 310,727
_________________________________________________________________


## Train

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

if not os.path.exists(model_path):
    os.mkdir(model_path)

# Validation 점수가 가장 좋은 모델만 저장합니다.
model_file_path = model_path + 'Epoch_{epoch:03d}_Val_{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath=model_file_path, monitor='val_loss', verbose=1, save_best_only=True)

# 10회 간 Validation 점수가 좋아지지 않으면 중지합니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=100)

# 모델을 학습시킵니다.
history = model.fit(
    x_data, y_data, 
    epochs=100, batch_size=32, validation_split=0.05, shuffle=False,
    callbacks=[checkpoint, early_stopping]
)


Epoch 1/100
2969/2969 [==============================] - ETA: 0s - loss: 2.4821
Epoch 00001: val_loss improved from inf to 2.07352, saving model to /content/drive/My Drive/dacon_voice/model/20200706_EfficientNetB7/Epoch_001_Val_2.074.hdf5
2969/2969 [==============================] - 873s 294ms/step - loss: 2.4821 - val_loss: 2.0735
Epoch 2/100
2969/2969 [==============================] - ETA: 0s - loss: 2.4769
Epoch 00002: val_loss did not improve from 2.07352
2969/2969 [==============================] - 858s 289ms/step - loss: 2.4769 - val_loss: 2.0815
Epoch 3/100
2969/2969 [==============================] - ETA: 0s - loss: 2.4600
Epoch 00003: val_loss did not improve from 2.07352
2969/2969 [==============================] - 856s 288ms/step - loss: 2.4600 - val_loss: 2.0751
Epoch 4/100
2969/2969 [==============================] - ETA: 0s - loss: 2.1104
Epoch 00004: val_loss improved from 2.07352 to 2.06981, saving model to /content/drive/My Drive/dacon_voice/model/20200706_EfficientNe

## Predict

In [9]:
x_test = np.load('/content/drive/My Drive/dacon_voice/data/np_save/stacked_melspec_test_96.npy')
print(x_test.shape)
# x_test = x_test.reshape(x_test.shape[0], x_test.shape[1] * x_test.shape[2], 1)

# 가장 좋은 모델의 weight를 불러옵니다.
weigth_file = glob(model_path + '*.hdf5')[-1]

# weight_file = '/content/drive/My Drive/dacon_voice/model/20200705_ResNet152V2/Epoch_005_Val_0.911.hdf5'
print(weight_file)
model.load_weights(weight_file)

# 예측 수행
y_pred = model.predict(x_test)

# 현재 시간 구하기
import time 
now = time.localtime()
now = "%04d%02d%02d_%02d%02d%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour+9, now.tm_min, now.tm_sec)
print(now)

# 예측 결과로 제출 파일을 생성합니다.
submission = pd.read_csv('/content/drive/My Drive/dacon_voice/data/submission.csv', index_col=0)
submission.loc[:, :] = y_pred
submission.to_csv('/content/drive/My Drive/dacon_voice/result/'+ now +'_submission.csv')

(10000, 96, 96, 3)
/content/drive/My Drive/dacon_voice/model/20200706_EfficientNetB7/Epoch_030_Val_0.861.hdf5
20200706_262806


## Transfer Learning

In [8]:
from keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from glob import glob

# 가장 좋은 모델의 weight를 불러옵니다.
# weigth_file = glob('model/*.hdf5')[-1]
weight_file = glob(model_path + '*.hdf5')[-1]
# weight_file = '/content/drive/My Drive/dacon_voice/model/Epoch_001_Val_2.083.hdf5'
print(weight_file)

# load h5
model = tf.keras.models.load_model(weight_file)

# model compile
# model.compile(loss=tf.keras.losses.KLDivergence(), optimizer='adam')

# Validation 점수가 가장 좋은 모델만 저장합니다.
model_file_path = model_path + 'Epoch_{epoch:03d}_Val_{val_loss:.3f}.hdf5'
checkpoint = ModelCheckpoint(filepath=model_file_path, monitor='val_loss', verbose=1, save_best_only=True)

# 10회 간 Validation 점수가 좋아지지 않으면 중지합니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=100)

# 모델을 학습시킵니다.
history = model.fit(
    x_data, y_data, 
    epochs=100, batch_size=32, validation_split=0.05, shuffle=False,
    callbacks=[checkpoint, early_stopping]
)


/content/drive/My Drive/dacon_voice/model/20200706_EfficientNetB7/Epoch_030_Val_0.861.hdf5
Epoch 1/100
2969/2969 [==============================] - ETA: 0s - loss: 0.3430
Epoch 00001: val_loss improved from inf to 0.91412, saving model to /content/drive/My Drive/dacon_voice/model/20200706_EfficientNetB7/Epoch_001_Val_0.914.hdf5
2969/2969 [==============================] - 864s 291ms/step - loss: 0.3430 - val_loss: 0.9141
Epoch 2/100
2969/2969 [==============================] - ETA: 0s - loss: 0.3275
Epoch 00002: val_loss improved from 0.91412 to 0.86528, saving model to /content/drive/My Drive/dacon_voice/model/20200706_EfficientNetB7/Epoch_002_Val_0.865.hdf5
2969/2969 [==============================] - 855s 288ms/step - loss: 0.3275 - val_loss: 0.8653
Epoch 3/100
2969/2969 [==============================] - ETA: 0s - loss: 0.3159
Epoch 00003: val_loss did not improve from 0.86528
2969/2969 [==============================] - 847s 285ms/step - loss: 0.3159 - val_loss: 0.8844
Epoch 4/100

KeyboardInterrupt: ignored